# About

This notebook counts the pixels per class in the three rasters generated by the iceplant classification model. 
The three rasters are non-overlapping and they cover the Santa Barbara County coast. These are:

* merged results of NAIP scenes covering the south-facing coast with crs 26910 (raster 0),
* merged results of NAIP scenes covering the west-facing coast with crs 26910 (raster 1),
* merged results of NAIP scenes with crs 26911 (raster 2).

The outputs are three csv files:
- *_pixel_counts_per_raster: the number of pixels per class in each raster
- *_combined_pixel_counts_per_raster: as previous file, but classes 2 and 3 (low NDVI and high NDWI) have been combined into a single one (non-vegetation)
- *_combined_pixel_counts_total: as previous file, but gives the total from three classes across all rasters.

In [ ]:
import os
import rioxarray as rioxr
import numpy as np
import pandas as pd

# Assuming repository's parent directory is the home directory
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'iceplant-detection-santa-barbara'))

In [ ]:
prefix = 'final_model'

In [ ]:
# -----------------------------------------------
# create directory for output csv files
out_dir = os.path.join(os.getcwd(),
                       'data',
                       'map',
                       'validation_data')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
out_dir = os.path.join(out_dir,
                       'pixel_counts')
if os.path.exists(out_dir) == False:
    os.mkdir(out_dir)
# -----------------------------------------------
# Load rasters
root = os.path.join(os.getcwd(),
                    'data',
                    'map',
                    'processing_results',
                    'final_model_preds_on_scenes')

fp = os.path.join(root, prefix+'_merged_crs26910_S.tif')
r_26910_S = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, prefix+'_merged_crs26910_W.tif')
r_26910_W = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, prefix+'_merged_crs26911.tif')
r_26911 = rioxr.open_rasterio(fp).squeeze()

rasters = [r_26910_S, r_26910_W, r_26911]

In [ ]:
# -----------------------------------------------
# count total number of pixels per category in combined rasters
n_nonice = []
n_ice = []
n_ground = []
n_water = []

for raster in rasters:
    n_nonice.append(len(np.where(raster == 0)[0]))
    n_ice.append(len(np.where(raster == 1)[0]))
    n_ground.append(len(np.where(raster == 2)[0]))
    n_water.append(len(np.where(raster == 3)[0]))
    

# -----------------------------------------------
raster_names = [prefix+'_merged_crs26910_S',
                prefix+'_merged_crs26910_W',
                prefix+'_merged_crs26911']

# -----------------------------------------------
# assemble and save dataframe with pixel counts
D = { 'n_other_veg': n_nonice,
      'n_ice': n_ice,
      'n_ground': n_ground, 
      'n_water': n_water}

df = pd.DataFrame(D)
df['raster'] = raster_names

out_path = os.path.join(out_dir,
                        prefix+'_pixel_counts_per_raster.csv')
                        
df.to_csv(out_path, index=False)

# -----------------------------------------------
# assemble and save dataframe with pixel counts, 
# water (high NDWI) and ground (low NDVI) are combined into same class

n_non_veg = [x+y for x,y in zip(n_ground,n_water)]

combined_df = pd.DataFrame({'n_other_veg': n_nonice,
      'n_ice': n_ice,
      'n_nonveg': n_non_veg})

combined_df['raster'] = raster_names

# save
out_path = os.path.join(out_dir,
                        prefix+'_combined_pixel_counts_per_raster.csv')
combined_df.to_csv(out_path, index=False)

# -----------------------------------------------
# total number of pixels across rasters in three classes: 
# other veg, iceplant and non-veg
total_df = pd.DataFrame([combined_df.sum(numeric_only=True)])

# save
out_path = os.path.join(out_dir,
                        prefix + '_combined_pixel_counts_total.csv')
total_df.to_csv(out_path, index=False)